# Addestramento modelli

In [ ]:
import warnings
import numpy as np
import pandas as pd
import os
import sys  
from datetime import datetime, timedelta

In [ ]:
warnings.filterwarnings("ignore")
sys.path.insert(0, "..")

In [ ]:
import utility

In [ ]:
from keras.models import load_model

In [ ]:
model_trained_base_path = "TrainingDirectory"
dataset_path = os.path.join("..", "Preprocessing", "Dataset", "dataset_training.csv")

In [ ]:
colors_model = {
    "modello1": "#FF7F50",
    "modello2": "#FFBF00",
    "modello3": "#6495ED",
    "modello4": "#93C572"
}

### Import dei dati

In [ ]:
df = utility.import_dataset(dataset_path, "2021", "2023")
display(df)

### OneHot encoding e normalizzazione

In [ ]:
df, scaler = utility.processing_dataset(df)

### Splitting in training, validation e test

In [ ]:
df_train, df_validation, df_test = utility.split_dataset(df, "2021-03", "2022-04")

In [ ]:
time_steps = 24 * 5
dim_other_feature = 24 + 7 + 12 + 1

In [ ]:
train = utility.windowed_dataset(df_train, time_steps)
validation = utility.windowed_dataset(df_validation, time_steps)
test = utility.windowed_dataset(df_test, time_steps)

### Result of training

In [ ]:
base_path = os.path.join(model_trained_base_path, "model4")
model = load_model(os.path.join(base_path, "weights-model.h5"))
history = utility.read_history(os.path.join(base_path, "history-model.json")) 
colore = colors_model["modello4"]

In [ ]:
utility.counter = 0
utility.plot_history(history, "#121212", colore, (10,5), base_path)

In [ ]:
y_pred_test = model.predict(utility.get_x(train, "CaricoTotale"))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df_prediction_train = utility.add_prediction(df_train, y_pred_test_inv, time_steps, "Carico totale predizione")
mape = utility.MAPE(df_prediction_train["Carico totale"], df_prediction_train["Carico totale predizione"])
mae = utility.MAE(df_prediction_train["Carico totale"], df_prediction_train["Carico totale predizione"])
print("MAPE training set:", mape)
print("MAE training set:", mae)

print()

y_pred_test = model.predict(utility.get_x(test, "CaricoTotale"))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df_prediction_test = utility.add_prediction(df_test, y_pred_test_inv, time_steps, "Carico totale predizione")
mape = utility.MAPE(df_prediction_test["Carico totale"], df_prediction_test["Carico totale predizione"])
mae = utility.MAE(df_prediction_test["Carico totale"], df_prediction_test["Carico totale predizione"])
print("MAPE test set:", mape)
print("MAE test set:", mae)

In [ ]:
utility.counter = 4
ft = "%Y-%m-%d %H:%M:%S"
start = datetime.strptime("2022-04-08 00:00:00", ft)
end = datetime.strptime("2022-04-28 23:00:00", ft)

fields = ["Carico totale", "Carico totale predizione"]
colors = ["#121212", colore]

title = None #"Confronto curva di carico e predizione da " + str(start) + " a " + str(end)
xlabel = None #"Ora"
ylabel = "Carico elettrico (W)"
dim = (15,7)
marker = False
lw = 3
utility.plot_distribution(df_prediction_test, fields, start, end, dim, title, xlabel, ylabel, base_path, marker, colors=colors, linewidth=lw)

##### Errori MAPE, MAE, RMSE sulle singole giornate

In [ ]:
df_error = utility.get_df_errors(df_prediction_test, "Carico totale", "Carico totale predizione")
utility.counter = 5

In [ ]:
df_sorted = df_error.sort_values(by=["mape"])
df_sorted

##### Errori MAPE, MAE, RMSE sui giorni della settimana

In [ ]:
df_error_day_of_week = utility.get_df_error_days_of_week(df_prediction_test, "Carico totale", "Carico totale predizione")
utility.counter = 8

In [ ]:
keys = df_error_day_of_week["DoW"]
values = np.round(df_error_day_of_week["mape"], 2)

ylabel = "MAPE (%)"
title = None #"Mean Absolute Percentage Error (MAPE) sui giorni della settimana"
color = colore
dim = (10,5)
linewidth = 1.3

utility.bar_plot(keys, values, ylabel, title, color, dim, linewidth, base_path)

In [ ]:
keys = df_error_day_of_week["DoW"]
values = np.round(df_error_day_of_week["mae"], 2)

ylabel = "MAE (W)"
title = None #"Mean Absolute Error (MAE) sui giorni della settimana"
color = colore
dim = (10,5)
linewidth = 1.3

utility.bar_plot(keys, values, ylabel, title, color, dim, linewidth, base_path)